# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [1]:
import urllib.request
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
To adapt the learning rate the Barzilai-Borwein method is used.

Try to understand each step of the learning algorithm and comment each line.


In [3]:
def learn_reg_ERM(X,y,lbda):
    max_iter = 200
    e  = 0.001
    alpha = 1.

    w = np.random.randn(X.shape[1]);
    for k in np.arange(max_iter):
        h = np.dot(X,w)
        l,lg = loss(h, y)
        print ('loss: {}'.format(np.mean(l)))
        r,rg = reg(w, lbda)
        g = np.dot(X.T,lg) + rg 
        if (k > 0):
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old)) 
        w = w - alpha * g
        if (np.linalg.norm(alpha * g) < e):
            break
        g_old = g
    return w

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss w.r.t $\textbf{h}$. (Note: The partial derivative of the hinge loss with respect to $\textbf{h}$  is $g_{i} = −y $ if $l_{i} > 0$, else $g_{i} = 0$)

In [9]:
def loss(h, y):
    # Compute the hinge loss
    l = np.maximum(0, 1 - y * h)

    # Compute the gradient of the hinge loss
    g = -y * (l > 0)

    return l, g

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [10]:
def reg(w, lbda):
    # Compute the L2 regularizer
    r = 0.5 * lbda * np.dot(w, w)

    # Compute the gradient of the L2 regularizer
    g = lbda * w

    return r, g

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [11]:
def predict(w, X):
    # Compute the dot product of w and X
    scores = np.dot(X, w)

    # Apply the sign function to the scores to get the class labels
    preds = np.sign(scores)

    return preds

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [12]:
# Convert the class labels from {0, 1} to {-1, 1}
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

# Set the regularization parameter
lbda = 0.01

# Train the linear model on the training data
w = learn_reg_ERM(X_train, y_train, lbda)

# Classify the test instances
y_pred_train = predict(w, X_train)
y_pred_test = predict(w, X_test)

# Compute the accuracy on the training data
accuracy_train = np.mean(y_pred_train == y_train)

# Compute the accuracy on the test data
accuracy_test = np.mean(y_pred_test == y_test)

print('Training accuracy: {}'.format(accuracy_train))
print('Test accuracy: {}'.format(accuracy_test))


loss: 1.2037555657722696
loss: 13.727515447171266
loss: 8.582762842923765
loss: 1813.1500000000021
loss: 233.71892836622834
loss: 582.2695968881267
loss: 229.94397355874963
loss: 1813.1499999999992
loss: 90.60274842322806
loss: 90.13140270238789
loss: 67.79506102321247
loss: 41.65662485510957
loss: 17.682107492623494
loss: 16.168115733363244
loss: 13.863881203169697
loss: 7.491990627366907
loss: 6.484886243970243
loss: 5.341126488144793
loss: 6.42074263735298
loss: 10.910239898909058
loss: 3.8596127056185474
loss: 3.223823567107332
loss: 5.216237631099078
loss: 6.354256329117704
loss: 2.7635677401108185
loss: 2.385300909942945
loss: 2.4432030479400746
loss: 2.3432185275449675
loss: 2.0848465498784754
loss: 1.9998076011802262
loss: 2.4877660766343466
loss: 3.0727968534707353
loss: 2.0224163844476
loss: 1.842244633309567
loss: 1.7647397443619046
loss: 1.8111385144218606
loss: 1.7118317779447672
loss: 1.6676616435350802
loss: 1.6503898848694676
loss: 1.7952738988275077
loss: 1.64361993126

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Train a linear model
linear_model = LogisticRegression()
linear_model.fit(X_train, y_train)
linear_pred_train = linear_model.predict(X_train)
linear_pred_test = linear_model.predict(X_test)

# Train a random forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred_train = rf_model.predict(X_train)
rf_pred_test = rf_model.predict(X_test)

# Train a decision tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_pred_train = dt_model.predict(X_train)
dt_pred_test = dt_model.predict(X_test)

# Compute the accuracy
linear_accuracy_train = accuracy_score(y_train, linear_pred_train)
linear_accuracy_test = accuracy_score(y_test, linear_pred_test)
rf_accuracy_train = accuracy_score(y_train, rf_pred_train)
rf_accuracy_test = accuracy_score(y_test, rf_pred_test)
dt_accuracy_train = accuracy_score(y_train, dt_pred_train)
dt_accuracy_test = accuracy_score(y_test, dt_pred_test)

print('Linear model training accuracy: ', linear_accuracy_train)
print('Linear model test accuracy: ', linear_accuracy_test)
print('Random forest training accuracy: ', rf_accuracy_train)
print('Random forest test accuracy: ', rf_accuracy_test)
print('Decision tree training accuracy: ', dt_accuracy_train)
print('Decision tree test accuracy: ', dt_accuracy_test)


Linear model training accuracy:  0.8375
Linear model test accuracy:  0.7700534759358288
Random forest training accuracy:  0.9375
Random forest test accuracy:  0.786096256684492
Decision tree training accuracy:  0.9375
Decision tree test accuracy:  0.6844919786096256
